In [5]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding

[nltk_data] Downloading package punkt_tab to /home/miza/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/miza/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/miza/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [96]:
import pandas as pd
import re
df = pd.read_csv("/home/miza/magisterka/Magisterka/src/data/bert.csv")
def preprocess_text(text):
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text = re.sub(r'[^\w\s]', '', text.strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    text = re.sub(r'\s*\b[a-zA-Z]\b\s*', ' ', text).strip()
    return text

def preprocess_cost(text):
    text = text.lower() 
    text=text.strip()  
    #text = re.sub(r'\s+', '', text)
    pattern = re.compile(r'(\d{1,3}\.\d{2})\d?')
    return pattern.findall(text)

def clean_token(token):
    for t in token: 
        if len(t) == 1:
            print(t)
            token.remove(t)
    return token
    

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_57386/2118193583.py:7: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text)


In [ ]:
import transformers
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained('dkleczek/bert-base-polish-uncased-v1')
bert_model = BertModel.from_pretrained('dkleczek/bert-base-polish-uncased-v1')


In [243]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

def data_preprocess(df):
    df['preprocessed_text'] = df['OCR_product'].apply(lambda x: preprocess_text(x))
    df['cost'] = df['OCR_product'].apply(lambda x: preprocess_cost(x))
    embedings = calculate_embedings(df['preprocessed_text'].tolist())
    le=LabelEncoder()
    categories =  le.fit_transform(df['Category'])
    return df, embedings, categories

def calculate_embedings(df):
    inputs = tokenizer(df, return_tensors='pt', truncation=True, max_length=32, padding='max_length')
    with torch.no_grad():
        outputs = bert_model(**inputs)
        #cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return outputs.pooler_output

In [244]:
df = pd.read_csv("/home/miza/magisterka/Magisterka/src/data/gpt_generated_data.csv")
df, embedings, categories = data_preprocess(df)


In [245]:
X, Y = embedings, categories
X_test, X_train, Y_test, Y_train = train_test_split(X,Y,test_size=0.7, random_state=42)

In [246]:
from xgboost import XGBClassifier
model = XGBClassifier()

In [247]:
model.fit(X_train,Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [248]:
preds = model.predict(X_test)
print(classification_report(Y_test, preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        58
           1       0.98      0.99      0.99       167
           2       1.00      1.00      1.00         7
           3       1.00      1.00      1.00        23
           4       0.98      0.98      0.98        55
           5       1.00      0.94      0.97        50

    accuracy                           0.99       360
   macro avg       0.99      0.99      0.99       360
weighted avg       0.99      0.99      0.99       360



NameError: name 'preds' is not defined